In [1]:
import tensorflow as tf
import ray
from ray.tune.registry import register_env
from ray.rllib.algorithms import dqn, ppo

from env import *
from model import *
from action_distribution import *

2023-03-14 15:40:11.061798: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 15:40:11.159620: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-14 15:40:11.629369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/abdul/miniconda3/lib/
2023-03-14 15:40:11.629425: W tensorflow/compiler/xla/stre

In [2]:
%load_ext autoreload
%autoreload 2

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-03-14 15:40:12.883044: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:12.888388: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:12.888613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:13.440369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:13.440510: I tensorflow/compiler/xla/stream_executo

True

In [3]:
if ray.is_initialized():
    ray.shutdown()
ray.init()

2023-03-14 15:40:15,145	INFO worker.py:1553 -- Started a local Ray instance.


Python version:,3.10.9
Ray version:,2.3.0


In [4]:
n_disks = 4
env_config = {
    'n_disks': n_disks,
    'max_n_steps': 10000 #2 ** (n_disks + 2)
}

ENV_NAME = 'hanoi_tower'
register_env(ENV_NAME, create_env)

config = ppo.PPOConfig()
config = config.training(lr=1e-3,
                         gamma=0.9,
                        #  vf_loss_coeff=0.5,
                         train_batch_size=512 * 2,
                         sgd_minibatch_size=256 * 1,
                         num_sgd_iter=16 * 1)
config = config.rollouts(num_rollout_workers=8,
                         num_envs_per_worker=1)
config = config.environment(ENV_NAME, 
                            env_config=env_config)
config = config.resources(num_gpus=1)
config = config.framework('tf')

model_config = {
    'n_hiddens': 128
}

config.model.update({
    'custom_model': MODEL_NAME,
    'custom_model_config': model_config,
    'custom_action_dist': ACTION_DIST_NAME,
    'vf_share_layers': True
})

config.seed = 19
agent = config.build()

2023-03-14 15:40:16,170	INFO algorithm_config.py:2899 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2023-03-14 15:40:16,180	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=2558270) 2023-03-14 15:40:19,769	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=2558270) 2023-03-14 15:40:19,769	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


(RolloutWorker pid=2558276) Model: "base_model"
(RolloutWorker pid=2558276) __________________________________________________________________________________________________
(RolloutWorker pid=2558276)  Layer (type)                   Output Shape         Param #     Connected to                     
(RolloutWorker pid=2558276) ==================================================================================================
(RolloutWorker pid=2558276)  obs_input (InputLayer)         [(None, 3, 4)]       0           []                               
(RolloutWorker pid=2558276)                                                                                                   
(RolloutWorker pid=2558276)  flatten (Flatten)              (None, 12)           0           ['obs_input[0][0]']              
(RolloutWorker pid=2558276)                                                                                                   
(RolloutWorker pid=2558276)  context1 (Dense)               (No

2023-03-14 15:40:21.943957: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:21.944139: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:21.944216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:21.945323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 15:40:21.945436: I tensorflow/compiler/xla/stream_executo

In [5]:
config.to_dict()

{'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'num_gpus': 1,
 'num_cpus_per_worker': 1,
 'num_gpus_per_worker': 0,
 '_fake_gpus': False,
 'num_trainer_workers': 0,
 'num_gpus_per_trainer_worker': 0,
 'num_cpus_per_trainer_worker': 1,
 'custom_resources_per_worker': {},
 'placement_strategy': 'PACK',
 'eager_tracing': False,
 'eager_max_retraces': 20,
 'tf_session_args': {'intra_op_parallelism_threads': 2,
  'inter_op_parallelism_threads': 2,
  'gpu_options': {'allow_growth': True},
  'log_device_placement': False,
  'device_count': {'CPU': 1},
  'allow_soft_placement': True},
 'local_tf_session_args': {'intra_op_parallelism_threads': 8,
  'inter_op_parallelism_threads': 8},
 'env': 'hanoi_tower',
 'env_config': {'n_disks': 4, 'max_n_steps': 10000},
 'observation_space': None,
 'action_space': None,
 'env_task_fn': None,
 'render_env': False,
 'clip_rewards': None,
 'normalize_actions': True,
 'clip_actions': False,
 'disable_env_checking': False,
 '

In [6]:
for _ in range(50):
    res = agent.train()
agent.save()

'/home/abdul/ray_results/PPO_hanoi_tower_2023-03-14_15-40-16nury8a66/checkpoint_000050'

In [7]:
env = create_env(env_config)
obs, _ = env.reset()

episode_reward = 0
done = False
while not done:
    action = agent.compute_single_action(obs, explore=False)
    print(obs, action)
    obs, reward, done, terminated, info = env.step(action)
    episode_reward += reward


[[1 2 3 4]
 [0 0 0 0]
 [0 0 0 0]] [0 1]
[[0 2 3 4]
 [0 0 0 1]
 [0 0 0 0]] [0 2]
[[0 0 3 4]
 [0 0 0 1]
 [0 0 0 2]] [1 2]
[[0 0 3 4]
 [0 0 0 0]
 [0 0 1 2]] [0 1]
[[0 0 0 4]
 [0 0 0 3]
 [0 0 1 2]] [2 0]
[[0 0 1 4]
 [0 0 0 3]
 [0 0 0 2]] [2 1]
[[0 0 1 4]
 [0 0 2 3]
 [0 0 0 0]] [0 1]
[[0 0 0 4]
 [0 1 2 3]
 [0 0 0 0]] [0 2]
[[0 0 0 0]
 [0 1 2 3]
 [0 0 0 4]] [1 2]
[[0 0 0 0]
 [0 0 2 3]
 [0 0 1 4]] [1 0]
[[0 0 0 2]
 [0 0 0 3]
 [0 0 1 4]] [2 0]
[[0 0 1 2]
 [0 0 0 3]
 [0 0 0 4]] [1 2]
[[0 0 1 2]
 [0 0 0 0]
 [0 0 3 4]] [0 1]
[[0 0 0 2]
 [0 0 0 1]
 [0 0 3 4]] [0 2]
[[0 0 0 0]
 [0 0 0 1]
 [0 2 3 4]] [1 0]
[[0 0 0 1]
 [0 0 0 0]
 [0 2 3 4]] [0 2]


In [8]:
obs, episode_reward

(array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 2, 3, 4]], dtype=int32),
 -16.0)

In [16]:
policy = agent.get_policy()
model = policy.model
model

In [21]:
model.flatten[0].base_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 observations (InputLayer)      [(None, 9)]          0           []                               
                                                                                                  
 fc_value_1 (Dense)             (None, 256)          2560        ['observations[0][0]']           
                                                                                                  
 fc_1 (Dense)                   (None, 256)          2560        ['observations[0][0]']           
                                                                                                  
 fc_value_2 (Dense)             (None, 256)          65792       ['fc_value_1[0][0]']             
                                                                                              

In [28]:
model.logits_and_value_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 logits (Dense)                 (None, 6)            1542        ['input_1[0][0]']                
                                                                                                  
 value_out (Dense)              (None, 1)            257         ['input_1[0][0]']                
                                                                                                  
Total params: 1,799
Trainable params: 1,799
Non-trainable params: 0
__________________________________________________________________________________________________


In [9]:
model.model.from_model.summary()

Model: "from_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ctx_input (InputLayer)      [(None, 32)]              0         
                                                                 
 from_logits (Dense)         (None, 3)                 99        
                                                                 
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.model.to_model.summary()

Model: "to_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 from_input (InputLayer)        [(None, 3)]          0           []                               
                                                                                                  
 ctx_input (InputLayer)         [(None, 32)]         0           []                               
                                                                                                  
 from_embedding (Dense)         (None, 32)           128         ['from_input[0][0]']             
                                                                                                  
 concatenate (Concatenate)      (None, 64)           0           ['ctx_input[0][0]',              
                                                                  'from_embedding[0][0]']  

-7.0

In [ ]:
env = HanoiTower(config=env_config)

In [ ]:
env.action_space.sample()

array([2, 1])

In [ ]:
obs = env.reset()
env.observation_space.contains(obs)

In [ ]:
obs

In [ ]:
env.step([0, 1])